In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 19.6 MB/s eta 0:00:00
161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 41.5s161.3 MiB [] 0% 30.1s161.3 MiB [] 0% 19.5s161.3 MiB [] 0% 13.0s161.3 MiB [] 1% 8.0s161.3 MiB [] 1% 6.5s161.3 MiB [] 2% 5.3s161.3 MiB [] 2% 4.9s161.3 MiB [] 3% 4.7s161.3 MiB [] 3% 4.5s161.3 MiB [] 4% 4.4s161.3 MiB [] 4% 4.3s161.3 MiB [] 5% 4.2s161.3 MiB [] 5% 4.1s161.3 MiB [] 5% 4.2s161.3 MiB [] 6% 4.2s161.3 MiB [] 7% 4.2s161.3 MiB [] 8% 4.1s161.3 MiB [] 8% 4.0s161.3 MiB [] 9% 3.9s161.3 MiB [] 10% 3.8s161.3 MiB [] 11% 3.8s161.3 MiB [] 12% 3.8s161.3 MiB [] 12% 3.7s161.3 MiB [] 13% 3.7s161.3 MiB [] 13% 3.6s161.3 MiB [] 14% 3.6s161.3 MiB [] 14% 3.7s161.3 MiB [] 15% 3.6s161.3 MiB [] 16% 3.5s161.3 MiB [] 17% 3.5s161.3 MiB [] 18% 3.4s161.3 MiB [] 18% 3.3s161.3 MiB [] 19% 3.3s161.3 MiB [] 20% 3.2s161.3 MiB [] 21% 3.1s161.3 MiB [] 22% 3.1s161.3 MiB [] 22% 3.0s161.3 MiB [] 23% 3.0s161.3 MiB [] 24% 3.0s161.3 MiB [] 24% 2.9s161.3 MiB [] 25% 2.9s161.3 MiB [] 26% 2.9s161.3 MiB [] 2

In [3]:
# パッケージのインストール
!pip install langchain langchain-google-genai pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25


In [4]:
!sudo apt-get install fonts-ipafont-gothic fonts-ipafont-mincho

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-ipafont-gothic fonts-ipafont-mincho
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 8,237 kB of archives.
After this operation, 28.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-ipafont-gothic all 00303-21ubuntu1 [3,513 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-ipafont-mincho all 00303-21ubuntu1 [4,724 kB]
Fetched 8,237 kB in 1s (5,663 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to f

In [5]:

# 環境変数の準備 (左端の鍵アイコンでGOOGLE_API_KEYを設定)
import os
from google.colab import userdata
import google.generativeai as genai

#os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [6]:
'''
#JSのパスを指定
js_path='/content/drive/MyDrive/WM最終課題/frame_mark_elements2.js'

#miniwobのhtmlを配置しているフォルダパス
miniwob_path = '/content/drive/MyDrive/WM最終課題/miniwob/html/miniwob/'
'''

#池田用
#JSのパスを指定
js_path='/content/drive/MyDrive/miniwob/frame_mark_elements2.js'

#miniwobのhtmlを配置しているフォルダパス
miniwob_path = '/content/drive/MyDrive/miniwob/html/miniwob/'

In [7]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# 安全性フィルタをなるべくなくす
safety_settings_nones: dict[HarmCategory, HarmBlockThreshold] = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    # HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY: HarmBlockThreshold.BLOCK_NONE, # ドキュメントにはあるが package にはない?
}

model = genai.GenerativeModel(
    "models/gemini-1.5-flash-002",
    #"models/gemini-2.0-flash-exp",
    generation_config=genai.GenerationConfig(
        temperature=0.1, # 出力を一貫させる観点からは 0 を指定したいが、RECITATION エラーが出やすく(主観)、retry で回避しづらくなるので低めの値にとどめている
        #max_output_tokens=1024,
        response_mime_type="application/json",
        #response_schema=schema,
    ),
    safety_settings=safety_settings_nones,
)

In [8]:
import base64
import io
import logging
import pkgutil
import re
from typing import Literal

import numpy as np
import PIL.Image
import playwright.async_api
import os

BID_ATTR = "bid"  # Playwright's default is "data-testid"
SOM_ATTR = "visFlg"
VIS_ATTR = "action_marks"

MARK_FRAMES_MAX_TRIES = 3

logger = logging.getLogger('miniwob')

class MarkingError(Exception):
    pass

def get_js_files_data(folder_path):
    js_files_data = None
    #for root, dirs, files in os.walk(folder_path):
        #for file in files:
            #if file.endswith(".js"):  # .jsファイルのみ対象
                #file_path = os.path.join(root, file)
    with open(folder_path, "r", encoding="utf-8") as f:
      js_files_data = f.read()  # ファイル名をキーに内容を保存
    return js_files_data

#事前にメモリに展開
js_frame_mark_elements = get_js_files_data(js_path)


async def _pre_extract(
    page: playwright.async_api.Page,
    tags_to_mark: Literal["all", "standard_html"] = "standard_html",
    lenient: bool = False,
):
    """
    pre-extraction routine, marks dom elements (set bid and dynamic attributes like value and checked)
    """

    # we can't run this loop in JS due to Same-Origin Policy
    # (can't access the content of an iframe from a another one)
    async def mark_frames_recursive(frame, frame_bid: str):
        assert frame_bid == "" or re.match(r"^[a-z][a-zA-Z]*$", frame_bid)
        logger.debug(f"Marking frame {repr(frame_bid)}")

        # mark all DOM elements in the frame (it will use the parent frame element's bid as a prefix)
        warning_msgs = await frame.evaluate(
            js_frame_mark_elements,
            [frame_bid, BID_ATTR, tags_to_mark],
        )
        # print warning messages if any
        for msg in warning_msgs:
            logger.warning(msg)

        # recursively mark all descendant frames
        for child_frame in frame.child_frames:
            # deal with detached frames
            if child_frame.is_detached():
                continue
            # deal with weird frames (pdf viewer in <embed>)
            child_frame_elem = child_frame.frame_element()
            if not child_frame_elem.content_frame() == child_frame:
                logger.warning(
                    f"Skipping frame '{child_frame.name}' for marking, seems problematic."
                )
                continue
            # deal with sandboxed frames with blocked script execution
            sandbox_attr = child_frame_elem.get_attribute("sandbox")
            if sandbox_attr is not None and "allow-scripts" not in sandbox_attr.split():
                continue
            child_frame_bid = child_frame_elem.get_attribute(BID_ATTR)
            if child_frame_bid is None:
                if lenient:
                    logger.warning("Cannot mark a child frame without a bid. Skipping frame.")
                    continue
                else:
                    raise MarkingError("Cannot mark a child frame without a bid.")
            await mark_frames_recursive(child_frame, frame_bid=child_frame_bid)

    # mark all frames recursively
    await mark_frames_recursive(page.main_frame, frame_bid="")

In [9]:
import asyncio
from playwright.async_api import async_playwright, Playwright

import nest_asyncio
nest_asyncio.apply()

import asyncio

# start browser, not to use 'with' context
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
#browser = await playwright.chromium.launch()
browser = await playwright.chromium.launch(
    headless=True,
    args=(
      [f"--window-size=500,320"]
    ),
    )

context = await browser.new_context()

def _activate_page_from_js(self, page):
   print('test')


await context.expose_binding(
            "browsergym_page_activated", lambda source: _activate_page_from_js(source["page"])
)
await context.add_init_script(
    r"""
window.browsergym_page_activated();
window.addEventListener("focus", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("focusin", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("load", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("pageshow", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("mousemove", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("mouseup", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("mousedown", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("wheel", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("keyup", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("keydown", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("input", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("touchstart", () => {window.browsergym_page_activated();}, {capture: true});
window.addEventListener("touchend", () => {window.browsergym_page_activated();}, {capture: true});
document.addEventListener("visibilitychange", () => {
    if (document.visibilityState === "visible") {
        window.browsergym_page_activated();
    }
}, {capture: true});
"""
        )



page = await context.new_page()
await page.expose_binding("pageURL", lambda source: 'unko')

#await page.expose_binding("pageURL", lambda source: source["page"].url)


await page.set_content("""

    <script>
      async function onClick() {
        document.querySelector('div').textContent = await window.pageURL();
      }
    </script>
    <button onclick="onClick()">Click me</button>
    <div></div>
    """)
await page.click("button")

print(await page.content())

# mark all frames recursively
await _pre_extract(page=page, tags_to_mark="standard_html", lenient=False)

body_html =  await page.content()

#print("HTML Source:\n")
#print(page_source)
print(body_html)

# end session
await context.close()
await browser.close()
await playwright.stop()

<html><head><script>
      async function onClick() {
        document.querySelector('div').textContent = await window.pageURL();
      }
    </script>
    </head><body><button onclick="onClick()">Click me</button>
    <div>unko</div>
    </body></html>
<html visflg="1" bid="0" action_marks="0"><head visflg="0" bid="1" action_marks="0"><script visflg="0" bid="2" action_marks="0">
      async function onClick() {
        document.querySelector('div').textContent = await window.pageURL();
      }
    </script>
    </head><body visflg="1" bid="3" action_marks="0"><button onclick="onClick()" visflg="1" value="" bid="4" action_marks="1">Click me</button>
    <div visflg="1" bid="5" action_marks="0">unko</div>
    </body></html>


In [10]:
import random

#miniwobから情報を取得
async def getinfo(
    #page: playwright.async_api.Page):
    page):

    (
    REWARD_GLOBAL,
    RAW_REWARD_GLOBAL,
    REWARD_REASON,
    DONE_GLOBAL,
    EPISODE_ID,
    TASK_READY,
    ) = await page.evaluate(
        r"""() => [WOB_REWARD_GLOBAL, WOB_RAW_REWARD_GLOBAL, WOB_REWARD_REASON, WOB_DONE_GLOBAL, WOB_EPISODE_ID, WOB_TASK_READY]"""
    )

    info = {
        "REWARD_GLOBAL": REWARD_GLOBAL,
        "RAW_REWARD_GLOBAL": RAW_REWARD_GLOBAL,
        "REWARD_REASON": REWARD_REASON,
        "DONE_GLOBAL": DONE_GLOBAL,
        "EPISODE_ID": EPISODE_ID,
        "TASK_READY": TASK_READY,
    }

    return info

#余分なHTMLを削る
async def removeDisplay(
    page):
    await page.evaluate(  # if error : core is not defined, make sure you started the miniwob http server at the right place miniwob-plusplus/miniwob/html
                    r"""
    let __display_ids = ['reward-display', 'click-canvas', 'sync-task-cover'];
    let __display_divs = {};
    let __query_div_hidden_copy = null;

    removeDisplay = function() {
      core.clearTimer();
      document.body.removeEventListener('click', core.canvasDrawClick);

      __query_div_hidden_copy = document.getElementById('query').cloneNode(true);
      //document.getElementById('query').innerHTML = '';

      for (i in __display_ids) {
        elem_id = __display_ids[i];
        elem = document.getElementById(elem_id);
        // remove elem from the document
        elem.remove();
        // but keep it stored somewhere to bring back later
        __display_divs[elem_id] = elem;
      }
    };

    bringBackDisplay = function() {
      document.getElementById('query').innerHTML = __query_div_hidden_copy.innerHTML;
      for (var elem_id in __display_divs){
        document.body.appendChild(__display_divs[elem_id]);
      }
      core.createDisplay();
    };

    core.endEpisode_legacy = core.endEpisode;
    core.startEpisodeReal_legacy = core.startEpisodeReal;
    core.getUtterance_legacy = core.getUtterance;

    core.getUtterance = function () {
      bringBackDisplay();
      utterance = core.getUtterance_legacy();
      removeDisplay();
      return utterance;
    };

    core.endEpisode = function(reward, time_proportional, reason){
      bringBackDisplay();
      core.endEpisode_legacy(reward, time_proportional, reason);
      removeDisplay();
    };

    core.startEpisodeReal = function() {
      bringBackDisplay();
      core.startEpisodeReal_legacy();
      removeDisplay();
    };

    removeDisplay();
    """
                )

#タスクを開始させる。
async def startTask(
    page):

    # start the task
    await page.evaluate(
                r"""
    Math.seedrandom({seed});
    core.EPISODE_MAX_TIME = {episode_max_time};
    core.startEpisodeReal();
    """.format(
                    #seed=0,#random.randint(0, 1000000),
                    seed=random.randint(0, 1000000),
                    episode_max_time=1000000,
                )
            )

    # wait for the Miniwob task to have status ready
    await page.wait_for_function(r"""() => WOB_TASK_READY""")

#目的の取得
async def getGoal(
    page):
    goal = await page.evaluate(r"""() => core.getUtterance()""")
    return goal


async def extract_elements_with_attribute(page , attribute, value):

          # 特定の属性と値に一致する要素を抽出
          elements = await page.query_selector_all(f'[{attribute}="{value}"]')
          #elements = page.locator(f'[{attribute}="{value}"]')

          # 各要素の内容を表示
          results = []
          for element in elements:

              # テキスト形式のJSON文字列に変換
              # JavaScriptを使ってすべての属性を取得
              details = {
              "tag_name": await element.evaluate("(el) => el.tagName"),
              "text": await element.inner_text(),
              "bid": await element.get_attribute("bid"),
              #"class": element.get_attribute("class"),
              }

              details["attributes"] = await element.evaluate(
                  "(el) => Array.from(el.attributes).reduce((acc, attr) => { acc[attr.name] = attr.value; return acc; }, {})"
              )

              serialized_data = json.dumps(details)
              results.append(serialized_data)

          return results


import json
import time
import pandas as pd
import google.generativeai as genai
from langchain_core.prompts import PromptTemplate
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from tqdm import tqdm


template ="""
あなたはHTML(環境）を解析し、次の要求を満たす行動を決定してください。
要求を理解した後に、SVGタグがある場合は特にひとつづ解析して判断してください。


#要求
{}

#環境
{}

#行動可能な情報
{}

#Action（フォーマット）
"action": 　, "bid":

"""


In [11]:
# メタプロンプトテンプレート
meta_prompt_template = """
You are a meta-controller. Your role is to orchestrate multiple experts (ReAct, CoT) to solve a given Web-based MiniWoB++ task. Each task provides an HTML environment and a goal. Your job is to decide which expert to call next and keep the insights up-to-date.

Information you have:
- Insights: {insights} (This includes the current task goal, relevant HTML elements, reasoning about what actions might help achieve the goal, and any advice from RCI.)
- Observations: {observations} (A list of previous results, actions taken, and any feedback or environment updates.)
- Expert History: {expert_history} (Which experts were called, in what order, and what they returned.)

Your available experts:
- ReAct: Decides concrete actions to interact with the HTML environment (e.g., click a button, hover over a menu item, scroll the page, input text into a field).
- CoT: Provides internal reasoning steps to better understand the current state, refine the plan, or identify what action might be needed next.

You must:
1. Determine the next expert to call based on the current goal, state, and any advice from RCI.
2. If “Insights” is empty, let CoT analyze it.
3. Ensure that the “next_expert” is always one of the valid expert names: "ReAct",or "CoT" Do not output "null" or any other value.

### Examples of valid output:

{{
  "next_expert": "ReAct"
}}

{{
  "next_expert": "CoT"
}}

Ensure that the output is strictly in the above JSON format with only the "next_expert" field.

Output in JSON:
{{
  "next_expert": "..."
}}
"""

# ReAct用プロンプト（HTML解析とアクション選択）
react_prompt_template = """
You are the ReAct expert.
Your task: Based on the given goal, the current HTML environment, and the actionable elements, decide which action to perform and which element (by bid) to select.

Respond to the following input, and avoid using the word "null" or providing a null-like response under any circumstances.

Consider that you can perform the following actions:
- "click": Click on an element.
- "hover": Hover over an element (e.g., to reveal a submenu).
- "scroll": Scroll the window to reveal elements (you must decide a direction or element if needed).
- "input": Type text into an input field (if the element is an <input> or editable field).
- "select": Select an option from a dropdown menu (if the element is a <select>).
- "dblclick": Double-click an element.
- "drag" and "drop": If needed, first choose "drag" an element, then "drop" it onto a target element. You may need to return both actions as separate steps or first choose a "drag" action and then another invocation to "drop".

You must return a single action and a single bid per invocation. If you need a complex sequence (like drag and then drop), the meta-controller can call ReAct multiple times.

#Current insights:
{insights}

#HTML:
{html}

#Actionable Elements:
{elements}

Output JSON (example):
{{
  "action": "click",
  "bid": "element_bid"
}}
"""

# CoT用プロンプト（内部推論、方針提案）
#SVD→SVG
cot_prompt_template = """
You are a specialized agent for analyzing HTML.
You are provided with the task goal, elements extracted from the HTML, and the HTML itself.
Carefully analyze the SVG images within the HTML and output the text, color, and size of each element in the SVG images.
Small text is defined as 12px or smaller, and large text is defined as 16px or larger.
If there are buttons, also output what each button represents.
Then, based on the goal and analysis results, define what kind of problem it is.

Finally, analyze the elements and suggest possible actions for the element to be selected from the following actions.
Please keep it concise for easier decision-making.

Consider that you can perform the following actions:
- "click": Click on an element.
- "hover": Hover over an element (e.g., to reveal a submenu).
- "scroll": Scroll the window to reveal elements (you must decide a direction or element if needed).
- "input": Type text into an input field (if the element is an <input> or editable field).
- "select": Select an option from a dropdown menu (if the element is a <select>).
- "dblclick": Double-click an element.
- "drag" and "drop": If needed, first choose "drag" an element, then "drop" it onto a target element. You may need to return both actions as separate steps or first choose a "drag" action and then another invocation to "drop".


Let’s think step by step.

# goal:
{goal}

#HTML:
{body_html}

#elements:
{elements}


Output in JSON:
{{
    "reasoning": "..."
}}
"""

cot_prompt_template3 = """
You are a specialized agent for analyzing HTML.
You are provided with the task goal, elements extracted from the HTML, and the HTML itself.
Carefully analyze the SVG images within the HTML and output the text, color, and size of each element in the SVG images.
Small text is defined as 12px or smaller, and large text is defined as 16px or larger.
If there are buttons, also output what each button represents.
Then, based on the goal and analysis results, define what kind of problem it is.
Finally, analyze the elements and suggest possible actions for the element to be selected.
Let’s think step by step.

# goal:
{goal}

#HTML:
{body_html}

#elements:
{elements}


Output in JSON:
{{
    "reasoning": "..."
}}
"""

cot_prompt_template2 = """
You are an expert agent parsing HTML.
Provides you with the goal of the task and the elements extracted from HTML and HTML.
Carefully analyze the SVG images in the HTML and output how many SVG images of what text, color, and size are in each. Small letters are defined as 12px or smaller Large letters are defined as 16px or larger.
If there are buttons, output what each button represents.
Then analyze which element to act on next.

# goal:
{goal}

#HTML:
{body_html}

#elements:
{elements}


Output in JSON:
{{
    "reasoning": "..."
}}
"""
# RCI用プロンプト（仮エージェント：情報整合性チェックなど）
rci_prompt_template = """
You are a Web Agent Observer.
Please review the web agent's activity history and errors listed below and advise the web agent what to watch out for and what to do next.
If you find the cause of the problem, you should also instruct the meta-agent what action to take next, along with the cause.

#Agent error:
{error}

#Prompts when errors occur
{prompts}

You must output the advice in the following JSON format exactly, with no additional text:

{{
  "advice": "Provide clear and actionable advice here."
}}
"""

In [ ]:
"""You are a
meta-controller.
Your role
is to
orchestrate
multiple
experts
(ReAct, CoT)"""

In [12]:
# RCIエージェント呼び出し関数
async def call_rci(model, expert_history, prompts, previousPrompt,error, max_retries=3):
    for attempt in range(1, max_retries + 1):
        rci_prompt = rci_prompt_template.format(
            history=json.dumps(expert_history, ensure_ascii=False, indent=2),
            #prompts="\n".join(prompts),
            prompts=previousPrompt['prompt'],
            error=error
        )
        prompts.append(rci_prompt)
        #previousPrompt = rci_prompt　ここは保存しない。
        response = None  # ここで初期化
        try:
            response = model.generate_content(
                rci_prompt,
                generation_config={
                    'temperature': 0.1,
                    'frequency_penalty':1.5,
                    #'max_output_tokens': 200,
                    'top_k': 5,
                    #'top_p': 0.95,
                    'response_mime_type': "application/json",
                }
            ).text.strip()

            # デバッグ用にレスポンスを出力
            print(f"RCI Response (Attempt {attempt}):\n{response}")

            # JSON部分のみを抽出するための正規表現
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                rci_json = json.loads(json_str)
                advice = rci_json.get("advice", "No advice provided.")
                return advice
            else:
                print("RCI response does not contain valid JSON.")

        except json.JSONDecodeError as e:
            print(f"Failed to parse RCI response as JSON on attempt {attempt}: {e}")
            print(f"RCI Response was: {response}")

        except Exception as e:
            print(f"An unexpected error occurred in call_rci on attempt {attempt}: {e}")
            print(f"RCI Response was: {response}")

        # リトライ前に短い待機を挿入
        await asyncio.sleep(6)

    # 全てのリトライが失敗した場合のデフォルトアドバイス
    print("All RCI attempts failed. Providing default advice.")
    return "RCI failed to provide valid advice. Please check the system and try again."

In [13]:
# メタモデル呼び出し関数
async def call_meta_model(model, meta_state, prompts,previousPrompt,metaCounter):
    meta_prompt = meta_prompt_template.format(
        insights=meta_state["insights"],
        observations=json.dumps(meta_state["observations"], ensure_ascii=False, indent=2),
        expert_history=json.dumps(meta_state["expert_history"], ensure_ascii=False, indent=2)
    )
    prompts.append(meta_prompt)
    previousPrompt['prompt'] = meta_prompt
    # print(f"Meta Prompt:\n{meta_prompt}")  # デバッグ用にプロンプトを出力
    meta_response = None
    try:
        meta_response = model.generate_content(
            meta_prompt,
            generation_config={
                'temperature': 0.01,
                'frequency_penalty':1.5,
                #'max_output_tokens': 200,
                #'top_k': 5,
                #'top_p': 0.95,
                'response_mime_type': "application/json",
            }
        ).text.strip()
        # print(f"Meta Response:\n{meta_response}")  # デバッグ用に応答を出力
        print(f"Meta Response:\n{meta_response}")  # デバッグ用に応答を出力

        meta_json = json.loads(meta_response)
        next_expert = meta_json.get("next_expert", "")
        if not next_expert or next_expert.lower() == "null":
            raise KeyError("next_expert is empty or null")
        if meta_json.get("updated_insights"):
            updated_insights = meta_json["updated_insights"]
            meta_state["insights"] = updated_insights
        # expert_history更新（例:次に呼ぶエキスパート記録）
        meta_state["expert_history"].append({"chosen_expert": next_expert})

        if metaCounter > 10:
            return "ReAct"
        else:
            return next_expert
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Meta model error: {e}")
        print(f"Response was: {meta_response}")
        # RCIを呼び出してアドバイスを取得
        error_message = str(e)
        advice = await call_rci(model, meta_state["expert_history"], prompts, error_message)
        print(f"RCI Advice: {advice}")
        # meta_stateにアドバイスを追加
        meta_state["insights"] += f"\nAdvice from RCI: {advice}"
        meta_state["expert_history"].append({"chosen_expert": "RCI"})
        if metaCounter > 10:
          return "ReAct"
        else:
          return "RCI"

In [14]:
# ReActエージェント呼び出し関数
async def call_react(model, insights, html, elements, prompts,previousPrompt, max_retries=3):
    for attempt in range(1, max_retries + 1):
        react_prompt = react_prompt_template.format(
            insights=insights,
            html=html,
            elements=json.dumps(elements, ensure_ascii=False, indent=2)
        )
        prompts.append(react_prompt)
        previousPrompt['prompt']=react_prompt
        response = None
        try:
            response = model.generate_content(
                react_prompt,
                generation_config={
                    'temperature': 0.01,
                    #'frequency_penalty':1.5,
                    #'max_output_tokens': 200,
                    #'top_k': 5,
                    #'top_p': 0.95,
                    'response_mime_type': "application/json",
                }
            ).text.strip()

            # デバッグ用にレスポンスを出力
            print(f"ReAct Response (Attempt {attempt}):\n{response}")

            # JSON部分のみを抽出するための正規表現
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                react_json = json.loads(json_str)
                action = react_json.get("action", "")
                bid = react_json.get("bid", "")
                return {"action": action, "bid": bid}
            else:
                print("ReAct response does not contain valid JSON.")

        except json.JSONDecodeError as e:
            print(f"Failed to parse ReAct response as JSON on attempt {attempt}: {e}")
            print(f"ReAct Response was: {response}")

        except Exception as e:
            print(f"An unexpected error occurred in call_react on attempt {attempt}: {e}")
            print(f"ReAct Response was: {response}")

        # リトライ前に短い待機を挿入
        await asyncio.sleep(1)

    # 全てのリトライが失敗した場合のデフォルトのアクション
    print("All ReAct attempts failed. Providing default action.")
    return {"action": "noop", "bid": ""}

In [15]:
# CoTエージェント呼び出し関数
async def call_cot(model, goal, body_html, ret, prompts,previousPrompt, max_retries=3):
    for attempt in range(1, max_retries + 1):
        cot_prompt = cot_prompt_template.format(
            goal=goal,
            body_html=body_html,
            elements=json.dumps(ret, ensure_ascii=False, indent=2),
        )
        prompts.append(cot_prompt)
        previousPrompt['prompt'] = cot_prompt
        response = None
        try:
            response = model.generate_content(
                cot_prompt,
                generation_config={
                    'temperature': 0.1,
                    'frequency_penalty':1.5,
                    #'max_output_tokens': 200,
                    'top_k': 5,
                    'top_p': 0.95,
                }
            ).text.strip()

            # デバッグ用にレスポンスを出力
            print(f"CoT Response (Attempt {attempt}):\n{response}")

            # JSON部分のみを抽出するための正規表現
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                cot_json = json.loads(json_str)
                reasoning = cot_json.get("reasoning", "")
                return reasoning
            else:
                print("CoT response does not contain valid JSON.")

        except json.JSONDecodeError as e:
            print(f"Failed to parse CoT response as JSON on attempt {attempt}: {e}")
            print(f"CoT Response was: {response}")

        except Exception as e:
            print(f"An unexpected error occurred in call_cot on attempt {attempt}: {e}")
            print(f"CoT Response was: {response}")

        # リトライ前に短い待機を挿入
        await asyncio.sleep(1)

    # 全てのリトライが失敗した場合のデフォルトの推論
    print("All CoT attempts failed. Providing default reasoning.")
    return "CoT failed to provide valid reasoning."

In [16]:
# colab need this for async
import time
from pydantic import BaseModel
from google.api_core.exceptions import TooManyRequests

import json
import nest_asyncio
nest_asyncio.apply()

import asyncio

# 保存用のDataFrame（空で作成）
df = pd.DataFrame()

# start browser, not to use 'with' context
from playwright.async_api import async_playwright
playwright = await async_playwright().start()

playwright.selectors.set_test_id_attribute(BID_ATTR)

#browser = await playwright.chromium.launch()
browser = await playwright.chromium.launch(
    headless=True,
    args=(
      [f"--window-size=768,512"]
    ),
    )

#動画とるならコメントアウトを解除
context =await browser.new_context(
    #record_video_dir="",
    #record_video_size={"height": 500, "width": 320},
    #record_video_size={"height": 768, "width": 1024},
    #record_video_size={"height": 100, "width": 200},
)

#context =await browser.new_context()

page = await context.new_page()

#await page.goto('file://' + miniwob_path + "count-shape.html")
#task_name = "click-button"
task_name = "click-color"
#task_name = "click-widget"
#task_name = "click-link"
#task_name = "click-dialog"
#task_name = "click-tab"
#task_name = "click-test-2"
#task_name = "click-test"
#task_name = "click-test-transfer"


await page.goto('file://' + miniwob_path + task_name + ".html")

await removeDisplay(page)
success=0
failed=0
test_cases = []
epoch = 50 #試行回数
for i in range(epoch):
    #--ここからループ
    await startTask(page)

    test_case = {
        'test_case_id': i + 1,
        'resultinfo': None,
        'prompts': []
    }
    prompts = test_case['prompts']
    previousPrompt = {'prompt':None}

    print (str(i+1) + "回目＝＝＝＝＝＝＝＝＝＝")
    info = await getinfo(page)
    #print(info)
    goal = await getGoal(page)

    #print(goal)

    # mark all frames recursively
    #await _pre_extract(page=page, tags_to_mark="standard_html", lenient=False)
    await _pre_extract(page=page, tags_to_mark="all", lenient=False)

    #HTMLを抽出
    body_html= await page.inner_html('body')
    #body_html=  await page.content()

    #actionlistを抽出
    ret = await extract_elements_with_attribute(page ,VIS_ATTR,"1")

    #print(ret)

    # LLMの準備
    llm = model

    meta_state = {
    "insights": "",
    "observations": {},
    "expert_history": []
    }

    meta_state["observations"]={
        "iteration": i,
        "info": info,
        "goal": goal,
        "html_snapshot": body_html,
        "elements": ret
    }
    resultinfo = {"DONE_GLOBAL":False}
    insights= await call_cot(llm, goal, body_html,ret,prompts,previousPrompt)
    meta_state["insights"] = insights
    # print("COT response:"+insights)
    meta_state["expert_history"].append("CoT")
    count=0
    metaCounter = 0
    reActCounter = 0
    while(resultinfo["DONE_GLOBAL"]==False and reActCounter <= 10):
        if count>5:
            advice = await call_rci(llm, meta_state["expert_history"],prompts,previousPrompt,error="The agent is unable to take action.")
            print(advice)
            meta_state["expert_history"].append("RCI")
            count=0
            time.sleep(6)
        #10回越えたら強制ReAct
        next_expert = await call_meta_model(model, meta_state,prompts,previousPrompt,metaCounter)

        #プロンプトの表示
        print(meta_state["insights"])
        time.sleep(6)
        if not next_expert or next_expert=="null":
            print("next_expertが空です。RCIを呼び出して再試行します。")
            meta_state["expert_history"].append("null")
            advice = await call_rci(model, meta_state["expert_history"], prompts,previousPrompt,"The meta-agent has failed to indicate the next action.")
            print(f"RCI Advice: {advice}")
            meta_state["insights"] += f"\nAdvice from RCI: {advice}"
            next_expert = "RCI"
        try:
            if next_expert == "ReAct":
                act = await call_react(model, meta_state["insights"], body_html, ret,prompts,previousPrompt)
                reActCounter = reActCounter + 1
                # 行動実行
                elem = page.get_by_test_id(act["bid"])
                action_type = act["action"].lower()
                if action_type == "click" or reActCounter > 5:
                    #5回越えたら強制終了
                    await elem.click(force=True)

                    #clickが連続する現象の調査用
                    if reActCounter > 5:
                        print(f"incompletePrompt:{previousPrompt['prompt']}")

                elif action_type == "hover":
                    await elem.hover()
                elif action_type == "scroll":
                    # スクロールの場合、特定要素までスクロールするか、ウィンドウ全体をスクロールするか決める必要あり。
                    # 要素があるならelem.scroll_into_view_if_needed()などを使う
                    await elem.scroll_into_view_if_needed()
                elif action_type == "input":
                    # 入力の際は act に "text" フィールドがあると想定する。
                    # 例えば: { "action": "input", "bid": "element_bid", "text": "Hello" }
                    if "text" in act:
                        await elem.fill(act["text"])
                    else:
                        # テキストが指定されていない場合はログ出力
                        meta_state["insights"] += f"\nAdvice select Action:No input text provided."
                        print("No input text provided.")
                elif action_type == "select":
                    # セレクトボックスの場合: { "action": "select", "bid": "element_bid", "option": "value" }
                    if "option" in act:
                        await elem.select_option(act["option"])
                    else:
                        print("No option provided for select.")
                elif action_type == "dblclick":
                    await elem.dblclick()
                elif action_type == "drag":
                    # dragの場合、次にdropアクションが別の呼び出しで来ることを想定
                    # Playwrightではdrag_and_drop()があるが、ここでは初回はdrag先がわからないため、meta_stateに記録するなどの工夫が必要
                    # ここでは簡略的にdrag先を既知としてる例 (実際にはmeta_stateでdrag元要素を記録し、別呼び出しでdrop先を処理)
                    meta_state["drag_source"] = act["bid"]
                elif action_type == "drop":
                    # dropの場合、 dragで記録したdrag_source要素からこのbidへドラッグ&ドロップを実行
                    if "drag_source" in meta_state:
                        source_elem = page.get_by_test_id(meta_state["drag_source"])
                        await source_elem.drag_to(elem)
                        del meta_state["drag_source"]
                    else:
                        print("No drag source recorded.")
                else:
                    print(f"Unknown action: {action_type}")

            elif next_expert == "CoT":
                reasoning = await call_cot(model,meta_state["observations"]["goal"],meta_state["observations"]["html_snapshot"],meta_state["observations"]["elements"],prompts,previousPrompt)
                # CoTの出力でinsightsを書き換え
                meta_state["insights"] = reasoning
                print(reasoning)
                metaCounter = metaCounter + 1

            elif next_expert == "RCI":
                advice = await call_rci(model, meta_state["insights"], meta_state["observations"],prompts,previousPrompt,"The agent is unable to take action.")
                print(f"RCI Advice: {advice}")
                meta_state["insights"] += advice

        except TooManyRequests as e:
            print(f"TooManyRequests error: {e}. Retrying after wait.")
            await asyncio.sleep(10)  # 必要に応じて待機時間を調整
            # Optionally, re-add the current iteration or handle it differently
            continue  # 次のイテレーションに進む

        except Exception as e:
            print(f"An unexpected error occurred: {e}.")
            continue  # 次のイテレーションに進む
        resultinfo = await getinfo(page)
        if (resultinfo["RAW_REWARD_GLOBAL"]==1):
            success+=1
        elif resultinfo['RAW_REWARD_GLOBAL']==-1:
            failed+=1
            print(f"failedPrompt:{previousPrompt['prompt']}")

        #報酬が０以外のとき
        if (resultinfo["RAW_REWARD_GLOBAL"]!=0):
          print("success:"+str(success))
          print("failed:"+str(failed))
          test_case['resultinfo'] = resultinfo
          test_case['prompts'] = prompts
          test_cases.append(test_case)

        time.sleep(5)
        count+=1
    time.sleep(3)
  #--ここまでループ
df = pd.DataFrame(test_cases)
print(df)
print(df.describe(include='all'))
print(success)

# RAW_REWARD_GLOBALが1の割合を計算
ratio = success / epoch #(success + failed)

# 結果を表示
print(f"Success Rate: {ratio:.2%}")


import pandas as pd
from datetime import datetime
import pytz

# 現在時刻をUTCで取得
utc_now = datetime.now(pytz.utc)

# 日本時間に変換
japan_timezone = pytz.timezone('Asia/Tokyo')
jst_now = utc_now.astimezone(japan_timezone)

# 現在の日付と時分秒を取得してファイル名に追加
current_time = jst_now.strftime("%Y%m%d_%H%M%S")
file_name = "池田_"+ task_name + f"_No.X_{current_time}.json"

# DataFrameをJSONファイルとして保存
df.to_json(file_name, orient='records', lines=True)


# end session
await context.close()
await browser.close()
await playwright.stop()

ストリーミング出力は最後の 5000 行に切り捨てられました。
- "scroll": Scroll the window to reveal elements (you must decide a direction or element if needed).
- "input": Type text into an input field (if the element is an <input> or editable field).
- "select": Select an option from a dropdown menu (if the element is a <select>).
- "dblclick": Double-click an element.
- "drag" and "drop": If needed, first choose "drag" an element, then "drop" it onto a target element. You may need to return both actions as separate steps or first choose a "drag" action and then another invocation to "drop".

You must return a single action and a single bid per invocation. If you need a complex sequence (like drag and then drop), the meta-controller can call ReAct multiple times.

#Current insights:
The goal is to click on the blue colored box. The HTML contains a div with id "query-color" and style attribute "background-color: blue;".  There is also a div with id "area" containing multiple colored boxes. One of them has the sty